In [3]:
import scipy.special

def is_lucky(length):
    if any(x in str(length) for x in "01235689"):
        return 0
    return 1

def fact(n):
    if n == 0:
        return 1
    res = 1
    while n > 1:
        res *= n
        n -= 1
    return res

def binomial(n, k):
    return fact(n) // (fact(k) * fact(n-k))

def possibilities(length):
    pair = []
    limit = length//2    #palindrom middle
    is_odd = length%2
    total = 0
    for x in range(0, limit+1):
        #Need to *2 because we will process only one side of palindrome
        if is_lucky(x*2+is_odd) == 1 or is_lucky(x*2) == 1 or is_lucky(length-x*2-is_odd) == 1 or is_lucky(length-x*2) == 1:
            pair.append(x)
    for i in pair:
        total += binomial(limit, i)
    print(length, pair, int(total))
    return total

for i in range(4,4445): #We hit the infinity barrier at 4444 number length 
    if is_lucky(i) == 1:
        possibilities(i)

4 [0, 2] 2
7 [0, 1, 2, 3] 8
44 [0, 2, 20, 22] 464
47 [0, 1, 2, 3, 20, 21, 22, 23] 4096
74 [0, 2, 15, 22, 35, 37] 18728400854
77 [0, 1, 2, 3, 15, 16, 22, 23, 35, 36, 37, 38] 75422540336
444 [0, 2, 22, 37, 185, 200, 220, 222] 3949534246514075237194788708873582383079842
447 [0, 1, 2, 3, 22, 23, 37, 38, 185, 186, 200, 201, 220, 221, 222, 223] 27912724205141181959450629580231778308791808
474 [0, 2, 15, 22, 37, 200, 215, 222, 235, 237] 54901863838970117759480286025527268246330592
477 [0, 1, 2, 3, 15, 16, 22, 23, 37, 38, 200, 201, 215, 216, 222, 223, 235, 236, 237, 238] 408867219021818229156357734782071193455492418
744 [0, 2, 22, 37, 135, 150, 222, 237, 335, 350, 370, 372] 731060835093247924457935113060011690556551334106988442540891669558161472551418643434436182653793845547130334
747 [0, 1, 2, 3, 22, 23, 37, 38, 135, 136, 150, 151, 222, 223, 237, 238, 335, 336, 350, 351, 370, 371, 372, 373] 3028768728981915246826884523329341022632256291149681236399608591744235816790585767417051756706213845557

In [1]:
""" i'll explain the logic to get the solution really fast
    so the current problem with trying to guess every possibility is that we need to go to 10^18 nth query
    the current method needs to find what is the previous query to find the next one (classic recursive)
    BUT we can see the problem in another way
    we can calculate exactly the amount of seven from a specific point thanks to combination and binomial factors
    
    we start like this (exemple) :
    we're looking for the 100th query, we first detect (thanks to range_limit) what is the length of the number
    10000 - range_limit[0] = 100 - 2 = 9998
     9998 - range_limit[1] = 98 - 8 = 9990
     9990 - range_limit[2] = 90 - 464 = negative
    so with this, we know that the length of the 100th query is a 44 length number and we also know that the
    number we're looking for is the 90th with this specific length
    
    what we do know is calculate when the last seven will appear, like this :
    
    
    
without the need to process every possibilities we need to directly know the amount of possibilities
    to do that there must be a formula because math exists
    lets have a case : we need to get the query 10000, with range_list we know that the result has a length of 74
    so we start with 4444444444444444444444444444444444444 (37)
    to get the next one we need to try if 4444444444444444444444444444444444447 is possible
    and so on and so on to get the 10000th query
    BUT because we know the amount of 7 needed to make the number a super lucky number we can calculate some things
    Imagine just having two 7s in a 5 length of 4s. The possibilities are
    4477
    4747
    4774
    7447
    7474
    7744
    and this can be calculate as binom(length, 2) which gives 6. Perfect
    So instead of creating every possibilities we might calculate directly the amount of possibilities from 
    the amount of 7.
    Let's retry the same exemple with the 37 length again.
    the 10000th query is calculate as follow :
    
    so inside the 74 length we need to find the 5430th possibility
    and then we do as follow : 
    we calculate the binom function step by step for every possibility of the placement we can do for the 7's
    """
import copy
import numpy as np
import scipy.special

class lucky():
    def __init__(self, query=0):
        #List of possible length   
        self.length_list = [4,7,44,47,74,77,444,447,474,477,744,747,774,777]
        #Amount of solutions by length. To use this I do query-range_list, once query becomes <=0 I go out the loop and do query+range_list
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246513022733692774166275217636196352, 27912724205136969367829202079257831965458432, 54901863838981370331031720163633347084419072, 408867219021824519558760381286789756865216512, 731060835093268059719935428380319894767978536241103340770302477220618973863638154274876419272070389261402112, 3028768728981208771138392332161427475712344495983372066026850069568299204087095837473068158977043396135223296, 383767495066681352637998366663200070705183074877734592607500475472627707939626649121930829298776144224727016144896, 1564887013153773591096691217957484965016329465244742414975904717557119142894155366820818937587853276546953480503296]
        #(this explanation is true for the first half of the number) List of amount of 4's and 7's for each length. pair_list works like this : pair_list[length_index][amount_of_4]. If you know the length and the amount of 4 you can find the amount of 7 with length-amount_of_4. Very useful   
        self.pair_4list = [[0,2], [0,1,2,3], [ 0, 2,20,22], [ 0, 1, 2, 3,20,21,22,23], [ 0, 2,15,22,35,37], [ 0, 1, 2, 3,15,16,22,23,35,36,37,38], [  0,  2, 22, 37,185,200,220,222], [  0,  1,  2,  3, 22, 23, 37, 38,185,186,200,201,220,221,222,223], [  0,  2, 15, 22, 37,200,215,222,235,237], [  0,  1,  2,  3, 15, 16, 22, 23, 37, 38,200,201,215,216,222,223,235,236,237,238], [  0,  2, 22, 37,135,150,222,237,335,350,370,372], [  0,  1,  2,  3, 22, 23, 37, 38,135,136,150,151,222,223,237,238,335,336,350,351,370,371,372,373], [  0,  2, 15, 22, 37,150,165,222,237,350,365,372,385,387], [  0,  1,  2,  3, 15, 16, 22, 23, 37, 38,150,151,165,166,222,223,237,238,350,351,365,366,372,373,385,386,387,388]]
        self.pair_7list = [[2,0], [3,2,1,0], [22,20, 2, 0], [23,22,21,20, 3, 2, 1, 0], [37,35,22,15, 2, 0], [38,37,36,35,23,22,16,15, 3, 2, 1, 0], [222,220,200,185, 37, 22,  2,  0], [223,222,221,220,201,200,186,185, 38, 37, 23, 22,  3,  2,  1,  0], [237,235,222,215,200, 37, 22, 15,  2,  0], [238,237,236,235,223,222,216,215,201,200, 38, 37, 23, 22, 16, 15,  3,  2,  1,  0], [372,370,350,335,237,222,150,135, 37, 22,  2,  0], [373,372,371,370,351,350,336,335,238,237,223,222,151,150,136,135, 38, 37, 23, 22,  3,  2,  1,  0], [387,385,372,365,350,237,222,165,150, 37, 22, 15,  2,  0], [388,387,386,385,373,372,366,365,351,350,238,237,223,222,166,165,151,150, 38, 37, 23, 22, 16, 15,  3,  2,  1,  0]]
        
        self.query = query
        self.solution = "" #store solutions
        self.cursor = 1 #store current solution index
        self.index = 0 #store current index of *_list variable
        self.target = 0
        self.limit = 0
        
    def get_solution(self, target):
        if target <= 0:
            print("lol k")
            return None
        if target > self.range_list[-1]:
            print("Number too big")
            return None
        
        self.index = 0
        while target-self.range_list[self.index] > 0:
            target -= self.range_list[self.index]
            self.index += 1
        
        reverse = 0
        if target-self.range_list[self.index]//2 > 0:
            target = self.range_list[self.index] - target + 1
            reverse = 1
        
        if target == 1:
            if reverse == 0:
                return "4"*self.length_list[self.index]
            if reverse == 1:
                return "7"*self.length_list[self.index]
        
        self.cursor = 1
        self.target = target
        self.solution = ""
        self.limit = self.length_list[self.index]//2
        res = [""]*self.limit
        self.generate(res, 0, 0, 0)
        
        if reverse == 0:
            return "".join(self.solution)
        res = []
        for i in self.solution:
            if i == "4":
                res += "7"
            else:
                res += "4"
        return "".join(res)
        
    def put(self, res, pos, val):
        res[pos] = val
        return res
    
    def generate(self, res, pos, num4, num7):
        if self.cursor > self.target:
            return
        if pos == self.limit:
            if self.length_list[self.index]%2 == 0:
                if num4*2 in self.length_list or num7*2 in self.length_list:
                    if self.cursor == self.target:
                        self.solution = res + res[::-1]
                    self.cursor += 1
            else:
                if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                    if self.cursor == self.target:
                        self.solution = res + ['4'] + res[::-1]
                    self.cursor += 1
                if num4*2 in self.length_list or num7*2 + 1 in self.length_list:
                    if self.cursor == self.target:
                        self.solution = res + ['7'] + res[::-1]
                    self.cursor += 1
        else:
            self.generate(self.put(copy.copy(res), pos, "4"), pos + 1, num4 + 1, num7)
            self.generate(self.put(copy.copy(res), pos, "7"), pos + 1, num4, num7 + 1)

yes = lucky()
for i in range(1,101):
    print(yes.get_solution(i), i)

4444
7777
4444444
4477744
4747474
4774774
7447447
7474747
7744477
7777777
44444444444444444444444444444444444444444444
44444444444444444444777744444444444444444444
44444444444444444447477474444444444444444444
44444444444444444447744774444444444444444444
44444444444444444474477447444444444444444444
44444444444444444474744747444444444444444444
44444444444444444477444477444444444444444444
44444444444444444744477444744444444444444444
44444444444444444744744744744444444444444444
44444444444444444747444474744444444444444444
44444444444444444774444447744444444444444444
44444444444444447444477444474444444444444444
44444444444444447444744744474444444444444444
44444444444444447447444474474444444444444444
44444444444444447474444447474444444444444444
44444444444444447744444444774444444444444444
44444444444444474444477444447444444444444444
44444444444444474444744744447444444444444444
44444444444444474447444474447444444444444444
44444444444444474474444447447444444444444444
44444444444444474744444444

False

In [4]:
import copy
import numpy as np
import scipy.special

class lucky_v2():
    def __init__(self, query=0):
        #List of possible length   
        self.length_list = [4,7,44,47,74,77,444,447,474,477,744,747,774,777]
        #Amount of solutions by length. To use this I do query-range_list, once query becomes <=0 I go out the loop and do query+range_list
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246513022733692774166275217636196352, 27912724205136969367829202079257831965458432, 54901863838981370331031720163633347084419072, 408867219021824519558760381286789756865216512, 731060835093268059719935428380319894767978536241103340770302477220618973863638154274876419272070389261402112, 3028768728981208771138392332161427475712344495983372066026850069568299204087095837473068158977043396135223296, 383767495066681352637998366663200070705183074877734592607500475472627707939626649121930829298776144224727016144896, 1564887013153773591096691217957484965016329465244742414975904717557119142894155366820818937587853276546953480503296]
        #(this explanation is true for the first half of the number) List of amount of 4's and 7's for each length. pair_list works like this : pair_list[length_index][amount_of_4]. If you know the length and the amount of 4 you can find the amount of 7 with length-amount_of_4. Very useful   
        self.pair_4list = [[0,2], [0,1,2,3], [ 0, 2,20,22], [ 0, 1, 2, 3,20,21,22,23], [ 0, 2,15,22,35,37], [ 0, 1, 2, 3,15,16,22,23,35,36,37,38], [  0,  2, 22, 37,185,200,220,222], [  0,  1,  2,  3, 22, 23, 37, 38,185,186,200,201,220,221,222,223], [  0,  2, 15, 22, 37,200,215,222,235,237], [  0,  1,  2,  3, 15, 16, 22, 23, 37, 38,200,201,215,216,222,223,235,236,237,238], [  0,  2, 22, 37,135,150,222,237,335,350,370,372], [  0,  1,  2,  3, 22, 23, 37, 38,135,136,150,151,222,223,237,238,335,336,350,351,370,371,372,373], [  0,  2, 15, 22, 37,150,165,222,237,350,365,372,385,387], [  0,  1,  2,  3, 15, 16, 22, 23, 37, 38,150,151,165,166,222,223,237,238,350,351,365,366,372,373,385,386,387,388]]
        self.pair_7list = [[2,0], [3,2,1,0], [22,20, 2, 0], [23,22,21,20, 3, 2, 1, 0], [37,35,22,15, 2, 0], [38,37,36,35,23,22,16,15, 3, 2, 1, 0], [222,220,200,185, 37, 22,  2,  0], [223,222,221,220,201,200,186,185, 38, 37, 23, 22,  3,  2,  1,  0], [237,235,222,215,200, 37, 22, 15,  2,  0], [238,237,236,235,223,222,216,215,201,200, 38, 37, 23, 22, 16, 15,  3,  2,  1,  0], [372,370,350,335,237,222,150,135, 37, 22,  2,  0], [373,372,371,370,351,350,336,335,238,237,223,222,151,150,136,135, 38, 37, 23, 22,  3,  2,  1,  0], [387,385,372,365,350,237,222,165,150, 37, 22, 15,  2,  0], [388,387,386,385,373,372,366,365,351,350,238,237,223,222,166,165,151,150, 38, 37, 23, 22, 16, 15,  3,  2,  1,  0]]
        
        self.query = query
        self.solution = "" #store solutions
        self.cursor = 1 #store current solution index
        self.index = 0 #store current index of *_list variable
        self.target = 0
        self.limit = 0
        
    def get_solution(self, target):
        if target <= 0:
            print("lol k")
            return None
        if target > self.range_list[-1]:
            print("Number too big")
            return None
        
        self.index = 0
        while target-self.range_list[self.index] > 0:
            target -= self.range_list[self.index]
            self.index += 1
        
        reverse = 0
        if target-self.range_list[self.index]//2 > 0:
            target = self.range_list[self.index] - target + 1
            reverse = 1
        
        if target == 1:
            if reverse == 0:
                return "4"*self.length_list[self.index]
            if reverse == 1:
                return "7"*self.length_list[self.index]
        
        self.cursor = 1
        self.target = target
        self.solution = ""
        self.limit = self.length_list[self.index]//2
        res = [""]*self.limit
        self.generate_v2(res, 0, 0, 0)
        
        if reverse == 0:
            return "".join(self.solution)
        res = []
        for i in self.solution:
            if i == "4":
                res += "7"
            else:
                res += "4"
        return "".join(res)
    
    def put(self, res, pos, val):
        res[pos] = val
        return res
    
    def is_possible(self, amount, space, values):
        if amount in values:
            if space == 1 and amount+1 not in values:
                return 0
            return 1
        idx = 0
        while amount > values[idx]:
            idx += 1
        if amount + space >= values[idx]:
            return 1
        return 0
    
    def generate_v2(self, res, pos, num4, num7):
        if self.cursor > self.target:
            return
        if pos == self.limit:
            save = self.cursor
            if self.length_list[self.index]%2 == 0:
                if num4*2 in self.length_list or num7*2 in self.length_list:
                    if self.cursor == self.target:
                        self.solution = res + res[::-1]
                    self.cursor += 1
            else:
                if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                    if self.cursor == self.target:
                        self.solution = res + ['4'] + res[::-1]
                    self.cursor += 1
                if num4*2 in self.length_list or num7*2 + 1 in self.length_list:
                    if self.cursor == self.target:
                        self.solution = res + ['7'] + res[::-1]
                    self.cursor += 1
            if save == self.cursor:
                print("too bad", "".join(res))
        else:
            if self.is_possible(num4, self.limit-pos, self.pair_4list[self.index]) == 1:
                self.generate_v2(self.put(copy.copy(res), pos, "4"), pos + 1, num4 + 1, num7)
            if self.is_possible(num7, self.limit-pos, self.pair_4list[self.index]) == 1:
                self.generate_v2(self.put(copy.copy(res), pos, "7"), pos + 1, num4, num7 + 1)

sol = lucky_v2()
for i in range(1,11):
    print(sol.get_solution(i), i)

4444 1
7777 2
4444444 3
4477744 4
4747474 5
4774774 6
7447447 7
7474747 8
7744477 9
7777777 10


0.0

In [59]:
import numpy as np
import scipy.special
import copy

def fact(n):
    if n == 0:
        return 1
    res = 1
    while n > 1:
        res *= n
        n -= 1
    return res

def binomial(n, k):
    return fact(n) // (fact(k) * fact(n-k))

def get_comb(length, pair, offset=0):
    for i in range(len(pair)):
        pair[i] -= offset
    comb = []
    for pos in range(1, length+1):
        total = 0
        for i in pair:
            if i <= pos:
                total += binomial(pos, i)
        comb.append(total)
    return comb

def recurs(res, cursor, pos, pair, depth):
    if depth > len(res):
        print("ERROR")
        return
    if cursor < 0:
        print("depth", depth, "// res", "".join(res[::-1]), "// cursor", cursor, "// pos", pos, "// pair", [i-depth for i in pair])
        return "".join(res[::-1])
    if cursor == 0:
        #print("before", res[::-1])
        idx = 0
        while pair[idx] < depth:
            idx += 1
        min_pair = pair[idx] - depth
        for i in range(min_pair):
            res[i] = "1"
        print("depth", depth, "// res", "".join(res[::-1]), "// cursor", cursor, "// pos", pos, "// pair", [i-depth for i in pair])
        return "".join(res[::-1])
    print("depth", depth, "// res", "".join(res[::-1]), "// cursor", cursor, "// pos", pos, "// pair", [i-depth for i in pair])
    length = len(res) - pos
    comb = get_comb(length, copy.copy(pair), depth)
    #print("comb", comb)
    idx = 0
    while idx < len(comb) and cursor >= comb[idx]:
        idx += 1
    print("idx", idx)
    res[idx] = "1"
    cursor -= comb[idx-1]
    print()
    return recurs(res, cursor, len(res) - idx, pair, depth + 1)

length = 444//2
pair = [0,2,22,37,185,200,220,222]
cursor = 3949534246513022733692774166275311787142112
cursor = 3949534246514075237194788708873582383079841
#print(get_comb(length, pair))
return
print(recurs(["0"] * length, cursor, 0, pair, 0))
return
all_res = []
limit = int(get_comb(length, pair)[-1])
for i in range(limit):
    all_res.append(recurs(["0"] * length, i, 0, pair, 0))
    print("\n-------------------",i+1,"\n")

print(limit, len(all_res))
for i in all_res:
    print(i)

SyntaxError: 'return' outside function (<ipython-input-59-0bcf74094b7b>, line 64)

In [37]:
import copy

class lucky_v3():
    def __init__(self):
        #List of possible length   
        self.length_list = [4, 7, 44, 47, 74, 77, 444, 447, 474, 477, 744, 747, 774, 777]
        #Amount of solutions by length. To use this I do query-range_list, once query becomes <=0 I go out the loop and do query+range_list
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246513022733692774166275217636196352, 27912724205136969367829202079257831965458432, 54901863838981370331031720163633347084419072, 408867219021824519558760381286789756865216512, 731060835093268059719935428380319894767978536241103340770302477220618973863638154274876419272070389261402112, 3028768728981208771138392332161427475712344495983372066026850069568299204087095837473068158977043396135223296, 383767495066681352637998366663200070705183074877734592607500475472627707939626649121930829298776144224727016144896, 1564887013153773591096691217957484965016329465244742414975904717557119142894155366820818937587853276546953480503296]
        #(this explanation is true for the first half of the number) List of the possible amount of 4's and 7's for each length.
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38], [0,2,22,37,185,200,220,222], [0,1,2,3,22,23,37,38,185,186,200,201,220,221,222,223], [0,2,15,22,37,200,215,222,235,237], [0,1,2,3,15,16,22,23,37,38,200,201,215,216,222,223,235,236,237,238], [0,2,22,37,135,150,222,237,335,350,370,372], [0,1,2,3,22,23,37,38,135,136,150,151,222,223,237,238,335,336,350,351,370,371,372,373], [0,2,15,22,37,150,165,222,237,350,365,372,385,387], [0,1,2,3,15,16,22,23,37,38,150,151,165,166,222,223,237,238,350,351,365,366,372,373,385,386,387,388]]
        
        self.limit = 0 #length//2, useful here and there
        self.seven_amount = 0 #the amount of seven needed to get the middle digit of odd length palindrom
        self.pair = [] #store self.pair_list[self.index]
        self.solution = "" #store solution
    
    def fact(self, n):
        if n == 0:
            return 1
        res = 1
        while n > 1:
            res *= n
            n -= 1
        return res

    def binomial(self, n, k):
        return fact(n) // (fact(k) * fact(n-k))

    def get_comb(self, length, offset=0):
        """ return the combination array depending of length and self.pair
        offset is used to decrease the value in pair array to create a better generic function
        
        combination (or comb) array works like this :
        for each index inside length we calculate the amount of possible combination that exists
        we simply use the binomial coeficient to get the values
        if parameters are wrong it stores 0 at current index
        
        Exemple : if length = 4 and pair = [2,3,4]
        The possibilities are : 0011, 0101, 0110, 0111, 1001, 1010, 1011, 1100, 1101, 1110, 1111 (11 possibilities)
        The comb array will return : [0,1,4,11]
        
        Why ? Let's explain step by step
        "0" in the current comb array means that there is no possibility to place our 1's within a length of 1
            "length of 1" is determine by the index of the value +1 ("0" is at index 0, so we add +1)
            
        "1" means that within a length of 2 we can have one possibility to place the 1's
            possibility is : 11 (we then add 0s at the start because we need to return a string of 4 digits)
            
        "4" means that there is 4 possibilities within a length of 3
            possibilities are : 011, 101, 110, 111
        
        "11" (the last value in array) gives the amount of all possibilites with the length of 4
        """
        comb = []
        for pos in range(1, length+1):
            total = 0
            for i in self.pair:
                i -= offset
                if i <= pos:
                    total += self.binomial(pos, i)
            comb.append(total)
        return comb
    
    def get_solution(self, query):
        index = 0
        while query - self.range_list[index] > 0: #we access the right index to access to the right parameters
            query -= self.range_list[index]
            index += 1
        
        query -= 1 #offset of 1 because there is a difference between input value and the one used in the algorithm
        self.limit = self.length_list[index]//2 #get half the length because palindrom
        self.seven_amount = 0
        self.pair = copy.copy(self.pair_list[index]) #we copy this because algorithm will alter the array
        self.solution = ["4"] * self.limit
        
        self.slp_algorithm(query) #everything
        
        if self.length_list[index]%2 == 0:
            self.solution = self.solution[::-1] + self.solution
        else: #we do this because if number is odd we need to fill the middle of the palindrom
            if self.seven_amount*2+1 in self.length_list or (self.limit-self.seven_amount)*2 in self.length_list:
                self.solution = self.solution[::-1] + ["7"] + self.solution
            else:
                self.solution = self.solution[::-1] + ["4"] + self.solution
        
        return "".join(self.solution)
    
    def slp_algorithm(self, cursor, pos=0, depth=0): #See top of file to better understand the algorithm
        if self.pair[0] - depth < 0: #update if the amount of seven is higher than the least pair value
            del self.pair[0]
        if cursor < 0: #a simple exception (it happens when all 7s are everywhere)
            return
        if cursor == 0: #if we're here, all remaining 7s to put will be placed at the beginning of the string
            for i in range(self.pair[0] - depth):
                self.solution[i] = "7"
                self.seven_amount += 1
            return
        
        comb = self.get_comb(self.limit - pos, depth) #get the comb array
        idx = 0
        while idx < self.limit - depth and cursor >= comb[idx]: #self.limit - pos = len(comb), so it's FASTER
            idx += 1
        self.solution[idx] = "7" #place the 7 at current index
        self.seven_amount += 1
        cursor -= comb[idx-1] #we update the value of cursor
        self.slp_algorithm(cursor, self.limit - idx, depth + 1)

yes = lucky_v3()
for i in range(1, 1):
    print(yes.get_solution(i), i)

yes.get_solution(sum(yes.range_list[:7]))

'777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777474777777777477777747777774747747477474747747744774777744474777744747477477744477777777774477747474477774477774747747477774477774474747774477777777774447774774747447777474447777477447747747474774747747477777747777774777777777474777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777'

In [27]:
#v2 = lucky_v2()
v3 = lucky_v3()

for i in range(1,1):
    res_v2 = v2.get_solution(i)
    res_v3 = v3.get_solution(i)
    if res_v2 != res_v3:
        print(i, res_v2, res_v3)
    else:
        if i%1000 == 0:
            print(i, "OK")

In [ ]:
class lucky_v4():
    def __init__(self):
        self.query = 0
        self.limit = 0 #length//2, useful here and there
        self.pair = [] #store the pair numbers
        self.seven_amount = 0 #the amount of seven needed to get the middle digit of odd length palindrom
        self.solution = "" #store solution
    
    def is_lucky(self, length):
        if any(x in str(length) for x in "01235689"):
            return 0
        return 1
    
    def get_pair(self, length):
        pair = []
        is_odd = length%2
        total = 0
        for x in range(0, limit+1):
            #Need to *2 because we will process only one side of palindrome
            if is_lucky(x*2+is_odd) == 1 or is_lucky(x*2) == 1 or is_lucky(length-x*2-is_odd) == 1 or is_lucky(length-x*2) == 1:
                pair.append(x)
        return pair
    
    def fact(self, n):
        if n == 0:
            return 1
        res = 1
        while n > 1:
            res *= n
            n -= 1
        return res

    def binomial(self, n, k):
        return fact(n) // (fact(k) * fact(n-k))
    
    def get_lucky_from_index(self, index):
        binary = "{0:b}".format(index+2)
        res = 0
        for idx, i in enumerate(binary[1:]):
            res += (4 if i == "0" else 7) * 10**(len(binary)-idx-2)
        return res
        
    def get_solution(self, query):
        index = 0
        while query > 0:
            total = 0
            length = self.get_lucky_from_index(index)
            self.limit = length // 2                  #get half the length because palindrom
            self.pair = self.get_pair(limit)
            for i in self.pair:
                total += self.binomial(limit, i)
            query -= total
            index += 1
        
        query += total
        query -= 1 #offset of 1 because there is a difference between input value and the one used in the algorithm
        self.seven_amount = 0
        self.solution = ["4"] * self.limit
        
        self.slp_algorithm(query) #everything
        
        if length%2 == 0:
            self.solution = self.solution[::-1] + self.solution
        else: #we do this because if number is odd we need to fill the middle of the palindrom
            if self.is_lucky(self.seven_amount*2+1) == 1 or self.is_lucky((self.limit-self.seven_amount)*2) == 1:
                self.solution = self.solution[::-1] + ["7"] + self.solution
            else:
                self.solution = self.solution[::-1] + ["4"] + self.solution
        
        return "".join(self.solution)

    def get_comb(self, length, offset=0):
        """ return the combination array depending of length and self.pair
        offset is used to decrease the value in pair array to create a better generic function
        
        combination (or comb) array works like this :
        for each index inside length we calculate the amount of possible combination that exists
        we simply use the binomial coeficient to get the values
        if parameters are wrong it stores 0 at current index
        
        Exemple : if length = 4 and pair = [2,3,4]
        The possibilities are : 0011, 0101, 0110, 0111, 1001, 1010, 1011, 1100, 1101, 1110, 1111 (11 possibilities)
        The comb array will return : [0,1,4,11]
        
        Why ? Let's explain step by step
        "0" in the current comb array means that there is no possibility to place our 1's within a length of 1
            "length of 1" is determine by the index of the value +1 ("0" is at index 0, so we add +1)
            
        "1" means that within a length of 2 we can have one possibility to place the 1's
            possibility is : 11 (we then add 0s at the start because we need to return a string of 4 digits)
            
        "4" means that there is 4 possibilities within a length of 3
            possibilities are : 011, 101, 110, 111
        
        "11" (the last value in array) gives the amount of all possibilites with the length of 4
        """
        comb = []
        for pos in range(1, length+1):
            total = 0
            for i in self.pair:
                i -= offset
                if i <= pos:
                    total += self.binomial(pos, i)
            comb.append(total)
        return comb
    
    def slp_algorithm(self, cursor, pos=0, depth=0): #See top of file to better understand the algorithm
        if self.pair[0] - depth < 0: #update if the amount of seven is higher than the least pair value
            del self.pair[0]
        if cursor < 0: #a simple exception (it happens when all 7s are everywhere)
            return
        if cursor == 0: #if we're here, all remaining 7s to put will be placed at the beginning of the string
            for i in range(self.pair[0] - depth):
                self.solution[i] = "7"
                self.seven_amount += 1
            return
        
        comb = self.get_comb(self.limit - pos, depth) #get the comb array
        idx = 0
        while idx < self.limit - depth and cursor >= comb[idx]: #self.limit - pos = len(comb), so it's FASTER
            idx += 1
        self.solution[idx] = "7" #place the 7 at current index
        self.seven_amount += 1
        cursor -= comb[idx-1] #we update the value of cursor
        self.slp_algorithm(cursor, self.limit - idx, depth + 1)

lucky_v4()

In [68]:
"{0:b}".format(3)

'11'

In [69]:
0 >   4 >   10 > 2
1 >   7 >   11 > 3
2 >  44 >  100 > 4
3 >  47 >  101 > 5
4 >  74 >  110 > 6
5 >  77 >  111 > 7
6 > 444 > 1000 > 8


False

In [138]:
def get_next_lucky(index):
    binary = "{0:b}".format(index+2)
    res = 0
    for idx, i in enumerate(binary[1:]):
        res += (4 if i == "0" else 7) * 10**(len(binary)-idx-2)
    return res
        
get_next_lucky(10)

744